# DB2-Salesforce connector: Tools mapping to authors

In [2]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'AuthorToolAssociation__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"purdue99272844vp8Z6qq0teuPQmirGaaPAt4",     
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [4]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')

import pandas as pd
import time
import datetime

from DB2SalesforceAPI import DB2SalesforceAPI

## Obtain tool information from DB2 

In [6]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))

# tool_authors
sql_query = "select * from jos_tool_authors"

toolauthors_df = pd.read_sql_query(sql_query, engine)

# display
toolauthors_df.head(2)

,toolname,revision,uid,ordering,version_id,name,organization
0,adept,16,4645,1,2,None,None
1,adept,16,3013,2,2,None,None


In [13]:
ta_df = toolauthors_df[['toolname', 'uid']].drop_duplicates()

## Obtain Salesforce IDs

In [7]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [19]:
# query the Salesforce IDs for contacts and citations. when updating junction objects, these IDs must be used

# get Salesforce ID for contacts
sf_user_ID_df = db_s.query_data('SELECT Id, nanoHUB_user_ID__c FROM Contact where nanoHUB_user_ID__c != NULL')

sf_user_ID_df['nanoHUB_user_ID__c'] = sf_user_ID_df['nanoHUB_user_ID__c'].astype('int')

# display
sf_user_ID_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000M1tx7AAB
{"id":"7505w00000M1tx7AAB","operation":"query","object":"Contact","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-05-15T04:42:12.000+0000","systemModstamp":"2020-05-15T04:42:13.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000M1tx7AAB","operation":"query","object":"Contact","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-05-15T04:42:12.000+0000","systemModstamp":"2020-05-15T04:42:22.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":217192,"retries":0,"totalProcessingTime":9101}
[Success] Bulk job completed successfully.


,Id,nanoHUB_user_ID__c
0,0035w000031Vsp1AAC,998
1,0035w000031Vsp2AAC,1683
2,0035w000031Vsp3AAC,1684


In [9]:
# get Salesforce ID for citations
sf_tool_df = db_s.query_data('SELECT Id, Tool_name__c FROM nanoHUB_tools__c')

# display
sf_tool_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000M1tu8AAB
{"id":"7505w00000M1tu8AAB","operation":"query","object":"nanoHUB_tools__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-05-15T04:38:01.000+0000","systemModstamp":"2020-05-15T04:38:02.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000M1tu8AAB","operation":"query","object":"nanoHUB_tools__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-05-15T04:38:01.000+0000","systemModstamp":"2020-05-15T04:38:05.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":1589,"retries":0,"totalProcessingTime":312}
[Success] Bulk job completed successfully.


,Id,Tool_name__c
0,a0s5w00000k5MOwAAM,ellipsom
1,a0s5w00000k5MOxAAM,hydrolab
2,a0s5w00000k5MOyAAM,huckel


## Match data with Salesforce format

In [20]:
# make sure all strings are lower cases
sf_tool_df['Tool_name__c'] = sf_tool_df['Tool_name__c'].astype('str').apply(str.lower)

In [21]:
# merge SF citation and contact IDs into user-citation DF
ct_tolink_df = pd.merge(ta_df, sf_tool_df, how='inner', left_on='toolname', right_on='Tool_name__c')\
                           .rename(columns={'Id':'SF_ID_tool'})
                                            
ct_tolink_df = pd.merge(ct_tolink_df, sf_user_ID_df, how='inner', left_on='uid', right_on='nanoHUB_user_ID__c')\
                           .rename(columns={'Id':'SF_ID_contact'})

# display
ct_tolink_df.head(2)

,toolname,uid,SF_ID_tool,Tool_name__c,SF_ID_contact,nanoHUB_user_ID__c
0,adept,4645,a0s5w00000k5MP7AAM,adept,0035w000034JHCCAA4,4645
1,adeptnpt,4645,a0s5w00000k5MWdAAM,adeptnpt,0035w000034JHCCAA4,4645


In [22]:
# create a new column for object name
ct_tolink_df['Name'] = ct_tolink_df.apply(lambda x: '%s_%s'%(x.nanoHUB_user_ID__c, x.Tool_name__c), axis=1)

In [23]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Name']         = ct_tolink_df['Name']
df_sf['Contact__c'] = ct_tolink_df['SF_ID_contact']
df_sf['Tool__c']       = ct_tolink_df['SF_ID_tool']
sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Name,4645_adept,4645_adeptnpt
Contact__c,0035w000034JHCCAA4,0035w000034JHCCAA4
Tool__c,a0s5w00000k5MP7AAM,a0s5w00000k5MWdAAM


## To Salesforce Sales Cloud CRM

In [24]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [25]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000M1txgAAB
[Success] CSV upload successful. Job ID = 7505w00000M1txgAAB
[Success] Closing job successful. Job ID = 7505w00000M1txgAAB


In [27]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_status())

{'apexProcessingTime': 0,
 'apiActiveProcessingTime': 5699,
 'apiVersion': 47.0,
 'columnDelimiter': 'COMMA',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'createdById': '0055w00000ArpYvAAJ',
 'createdDate': '2020-05-15T04:42:52.000+0000',
 'externalIdFieldName': 'Name',
 'id': '7505w00000M1txgAAB',
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'numberRecordsFailed': 0,
 'numberRecordsProcessed': 2133,
 'object': 'AuthorToolAssociation__c',
 'operation': 'upsert',
 'retries': 0,
 'state': 'JobComplete',
 'systemModstamp': '2020-05-15T04:43:08.000+0000',
 'totalProcessingTime': 7174}
